In [218]:
from tools.lang_to_csv import load_file

In [219]:
dialogues = open("dialogues.txt", "r").read()

In [220]:
import re

In [221]:
DATABASE_SEP = """

################################################################
##########################  DATABASE  ##########################
################################################################

"""

In [222]:
events, database = dialogues.split(DATABASE_SEP)

In [223]:
len('################################################################')

64

In [224]:
from collections import defaultdict

def extract_page_dmk(evtext, result, regex=r"_{64}\n\n_{64}\nPAGE (.*)\n_{64}\n"):
    pages = {}
    raw_pages = re.split(regex, evtext)
    raw_pages.pop(0)
    itp = iter(raw_pages)
    for pnum, ptext in zip(itp, itp):
        ptext = re.sub("_{64}\n*", "", ptext)
        raw_text = re.split(r'(-{49}|\+{49})\[(.*)\]', ptext)
        raw_text.pop(0)
        ittext = iter(raw_text)
        for sep, tid, value in zip(ittext, ittext, ittext):
            if sep[0] == '+':
                options = value.strip().split('\n') 
                result[tid] = options
            else:
                if '___________________' in value:
                    print(tid, repr(value))
                result[tid] = value.lstrip()
            pages[tid] = pnum
    return pages
    

def extract_dialogs_dmk(maptext):
    dialogs = {}
    event_ids = {}

    raw_maps = re.split(r"#{64}\nMAP (.*)\n#{64}\n", maptext)
    start = raw_maps.pop(0)

    itmap = iter(raw_maps)
    for num, text in zip(itmap, itmap):
        num = int(num)
        dialogs[num] = {} 
        raw_events = re.split(r"/{64}\nEVENT (.*)   (.*)   \(*(.*)\)\n", text.strip())
        raw_events.pop(0)
        itev = iter(raw_events)
        for evnum, evname, evpos, evtext in zip(itev, itev, itev, itev):
            evnum = int(evnum)
            dialogs[num][evnum] = {}
            event_ids[(num, evnum)] = (evname, evpos)
            extract_page_dmk(evtext, dialogs[num][evnum], r"_{64}\n\n_{64}\nPAGE (.*)\n_{64}\n")
    return dialogs, event_ids, start

dialogs, event_ids, start = extract_dialogs_dmk(events)

In [226]:
def extract_dict_dmk(text, result):
    raw_kv = re.split(r"_*\[(.*)\]\n", text)
    raw_kv.pop(0)
    it = iter(raw_kv)
    for key, value in zip(it, it):
        result[key] = re.sub(r"(_{64})?\n*", "", value)

def extract_listdict_dmk(text, result):
    raw_kv = re.split(r"_*\[(.*)\]\n", text)
    raw_kv.pop(0)
    it = iter(raw_kv)
    for key, value in zip(it, it):
        result[key] = re.sub(r"(_{64})?\n*$", "", value).split("\n")
        if len(result[key]) == 1:
            result[key].append("")
        if len(result[key]) == 3 and not result[key][2]:
            result[key].pop()

def extract_system_dmk(text, result):
    raw_sys = re.split(r"/{64}\nSYSTEM (.*)\n_{64}\n", text.strip())
    raw_sys.pop(0)
    it = iter(raw_sys)
    for key, value in zip(it, it):
        result[key] = {}
        extract_dict_dmk(value, result[key])
    
DMK_DATABASE_FUNC = {
    "ACTORS": "dict",
    "ANIMATIONS": "dict",
    "ARMORS": "listdict",
    "CLASSES": "dict",
    "COMMON EVENTS": "page",
    "ENEMIES": "dict",
    "ITEMS": "listdict",
    "MAP INFOS": "dict",
    "SKILLS": "listdict",
    "STATES": "dict",
    "SYSTEM": "system",
    "TILESETS": "dict",
    "TROOPS": "none",
    "WEAPONS": "listdict",
}

def extract_database_dmk(database):
    db = {}
    common_events = {}
    raw_db = re.split(r"#{64}\n(.*)\n#{64}\n", database)
    start = raw_db.pop(0)
    dit = iter(raw_db)
    for category, ctext in zip(dit, dit):
        db[category] = {}
        mode = DMK_DATABASE_FUNC[category] 
        if mode == "dict":
            extract_dict_dmk(ctext, db[category])
        if mode == "listdict":
            extract_listdict_dmk(ctext, db[category])
        if mode == "system":
            extract_system_dmk(ctext, db[category])
        if mode == "page":
            common_events = extract_page_dmk(ctext, db[category], r"\n_{64}\nCOMMON EVENT (.*)\n_{64}\n")
    return db, common_events
db, common_events = extract_database_dmk(database)

In [119]:


# Others: $title_lang, $menu_lang, $end_lan, $MOM, $MOMHINT, $journal_lang 


In [269]:
variables, header = load_file("Data/mods/translation/en/lang.rb")
variables_pt, header_pt = load_file("Data/mods/translation/ptbr/lang-Copy1.rb")

def convert_events(lang, events_lang):
    for num, map in events_lang.items():
        if num == 0:
            continue
        lang[num] = {}
        for evnum, events in map.items():
            lang[num][evnum] = {}
            for x, sub1 in events.items():
                for y, sub2 in sub1.items():
                    if isinstance(sub2, str):
                        for z, value in enumerate(sub2.split('------------------------------------------\n')):
                            lang[num][evnum][(x, y, z)] = value
                    else:
                        lang[num][evnum][(x, y)] = sub2[0]

def convert_common_events(values):
    result = {}
    for a, events in values.items():
        for x, sub1 in events.items():
            for y, sub2 in sub1.items():
                if isinstance(sub2, str):
                    for z, value in enumerate(sub2.split('------------------------------------------\n')):
                        result[(a, x, y, z)] = value
                else:
                    result[(a, x, y)] = sub2[0]
    return result
                        
def convert_lang(lang_rb):
    lang = {}
    for key, values in lang_rb.items():
        lang[key] = {}
        if key == "g__events_lang":
            convert_events(lang[key], values)
            lang["g__common_events"] = convert_common_events(values[0])
        else:
            lang[key] = values
    return lang
en = convert_lang(variables)
ptbr = convert_lang(variables_pt)


In [270]:
def find_dict_conversion(result, dmk, lang):
    used = set()
    for dkey, dvalue in dmk.items():
        total = partial = None
        for ekey, evalue in lang.items():
            if evalue == dvalue:
                if ekey in used:
                    partial = ekey
                else:
                    total = ekey
                    used.add(ekey)
                    break
        if not total and not partial:
            print(dkey, dvalue, "not found in", dmk)
        elif total:
            result[total] = dkey
        elif partial:
            print(dkey, partial, "duplicated")

def convert_map_events(conversion, dialogs, lang):
    matching_maps = set(dialogs.keys()) & set(lang.keys())
    for num in matching_maps:
        conversion[num] = {}
        dml = set(dialogs[num].keys()) - set(lang[num].keys())
        if dml:
            print("Events in dialog, but not in lang: ", num, dml)
        lmd = set(lang[num].keys()) - set(dialogs[num].keys())
        if lmd:
            print("Events in lang, but not in dialog: ", num, lmd)
        for evnum in set(dialogs[num].keys()) & set(lang[num].keys()):
            conversion[num][evnum] = {}
            find_dict_conversion(conversion[num][evnum], dialogs[num][evnum], lang[num][evnum])
            
DATABASE_DMK_LANG = {
    "ACTORS": "g__actors_lang",
    "ANIMATIONS": "g__animations_lang",
    "ARMORS": "g__armors_lang",
    "CLASSES": "g__classes_lang",
    "COMMON EVENTS": "g__common_events",
    "ENEMIES": "g__enemies_lang",
    "ITEMS": "g__items_lang",
    "MAP INFOS": "g__mapinfos_lang",
    "SKILLS": "g__skills_lang",
    "STATES": "g__states_lang",
    "SYSTEM": {
        "WORDS": "g__system_words",
        "ELEMENTS": "g__system_elements",
    },
    "TILESETS": "g__tilesets_lang",
    "TROOPS": "g__troops_lang",
    "WEAPONS": "g__weapons_lang"
}
            
conversion = {}
conversion["MAPS"] = {}
convert_map_events(conversion["MAPS"], dialogs, en["g__events_lang"])
for key, varname in DATABASE_DMK_LANG.items():
    conversion[key] = {}
    mode = DMK_DATABASE_FUNC[key] 
    if mode in ("dict", "listdict", "page"):
        find_dict_conversion(conversion[key], db[key], en[varname])
    if mode == "system":
        conversion[key]["WORDS"] = {}
        find_dict_conversion(conversion[key]["WORDS"], db[key]["WORDS"], en[varname["WORDS"]])
        conversion[key]["ELEMENTS"] = {
            int(x.split(":")[1]): x for x in db[key]["ELEMENTS"]
        }



In [271]:
lang = ptbr

def translate_map_events(translation, conversion, lang):
    for num, maps in lang.items():
        if num == 0:
            continue
        translation[num] = {}
        for evnum, events in maps.items():
            translation[num][evnum] = {}
            for tid, text in events.items():
                if tid not in conversion[num][evnum]:
                    print(num, evnum, tid, text)
                else:
                    translation[num][evnum][conversion[num][evnum][tid]] = text
                    
def translate_dict(translation, conversion, lang, log=None):
    for tid, text in lang.items():
        if tid not in conversion:
            if text and text != ["", ""]:
                print(log, tid, "not found", text)
        else:
            translation[conversion[tid]] = text

def translate_list(translation, conversion, lang, log=None):
    for tid, text in enumerate(lang):
        if tid not in conversion:
            if text and text != ["", ""]:
                print(log, tid, "not found", text)
        else:
            translation[conversion[tid]] = text
            

translations = {}
translations["MAPS"] = {}
translate_map_events(translations["MAPS"], conversion["MAPS"], lang["g__events_lang"])
for key, varname in DATABASE_DMK_LANG.items():
    translations[key] = {}
    mode = DMK_DATABASE_FUNC[key] 
    if mode in ("dict", "listdict", "page"):
        translate_dict(translations[key], conversion[key], lang[varname], log=key)
    if mode == "system":
        translations[key]["WORDS"] = {}
        translate_dict(translations[key]["WORDS"], conversion[key]["WORDS"], lang[varname["WORDS"]], log=key)
        translations[key]["ELEMENTS"] = {}
        translate_list(translations[key]["ELEMENTS"], conversion[key]["ELEMENTS"], lang[varname["ELEMENTS"]], log=key)


In [272]:
start

'DMK 4.9.1\n\n'

In [273]:
from itertools import groupby

def create_page(result, events, index=2, formatter=lambda x: "PAGE {}".format(x + 1)):
    new_keys = {
        tuple(map(int, event.split(":")[1:])): event
        for event in events.keys()
    }
    for page, group in groupby(sorted(new_keys.keys()), key=lambda x: x[index]):
        result.append("_" * 64)
        result.append(formatter(page))
        result.append("_" * 64)
        for new_key in group:
            tid = new_keys[new_key]
            event = events[tid]
            if isinstance(event, str):
                result.append('-' * 49 + '[{}]'.format(tid))
                if event[-1] == "\n":
                    event = event[:-1]
                result.append(event)
            else:
                result.append('+' * 49 + '[{}]'.format(tid))
                result.extend(event)
        result.append("_" * 64)
        result.append("")

def create_dict(result, values, tuplekeys=True):
    if tuplekeys:
        new_keys = {
            tuple(map(int, did.split(":")[1:])): did
            for did in values.keys()
        }
        order = sorted(new_keys.keys())
    else:
        new_keys = {did: did for did in values.keys()}
        order = new_keys
    for new_key in order:
        old_key = new_keys[new_key]
        head = "[{}]".format(old_key).rjust(64, '_')
        result.append(head)
        value = values[old_key]
        if isinstance(value, str):
            result.append(value)
        else:
            result.extend(value)
    result.append("_" * 64)
        
result = [start[:-1] if start and start[-1] == "\n" else start]
for num, maps in translations["MAPS"].items():
    result.append("#" * 64)
    result.append("MAP {}".format(num))
    result.append("#" * 64)
    result.append("")
    result.append("")
    for evnum, events in maps.items():
        result.append("/" * 64)
        result.append("EVENT {}   {}   ({})".format(evnum, *event_ids[num, evnum]))
        result.append("_" * 64)
        result.append("")
        create_page(result, events, index=2, formatter=lambda x: "PAGE {}".format(x + 1))
        result.append("")
        result.append("")
    result.append("")
    
result.append(re.sub("^\n", "", DATABASE_SEP))

for key, varname in DATABASE_DMK_LANG.items():
    result.append("#"*64)
    result.append(key)
    result.append("#"*64)
    mode = DMK_DATABASE_FUNC[key] 
    if mode in ("dict", "listdict"):
        create_dict(result, translations[key])
    if mode == "page":
        result.append("")
        create_page(result, translations[key], index=0, formatter=lambda x: "COMMON EVENT {}".format(x))
        result.pop()
    if mode == "system":
        result.append("")
        result.append("")
        result.append("/" * 64)
        result.append("SYSTEM WORDS")
        result.append("_" * 64)
        result.append("")
        create_dict(result, translations[key]["WORDS"], tuplekeys=False)
        result.append("")
        result.append("")
        result.append("/" * 64)
        result.append("SYSTEM ELEMENTS")
        result.append("_" * 64)
        result.append("")
        create_dict(result, translations[key]["ELEMENTS"])
        
    
    result.append("")
    result.append("")
    result.append("")
result.append("")

with open('diag2.txt', 'w') as f:
    f.write('\n'.join(result))
    #f.write(database)

In [137]:
# ToDo: match lang[0] to Common Events

In [206]:
x = "[asdasd]"
x.rjust(64, '_')

'________________________________________________________[asdasd]'